In [ ]:
import os
import json
from pprint import pprint
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

from dataservice.util.data_import.utils import (
    read_json, 
    write_json, 
    cols_to_lower, 
    dropna_rows_cols,
    reformat_column_names,
    extract_uncompressed_file_ext
)

DATA_DIR = '/Users/singhn4/Projects/kids_first/data/Chung'
DBGAP_DIR = os.path.join(DATA_DIR, 'dbgap')
MANIFESTS_DIR = os.path.join(DATA_DIR, 'manifests')

In [ ]:
# Create study
study = {
    'data_access_authority': 'dbGaP',
    'study_id': 'phs001110',
    'study_version': 'v1.p1',
    'study_name': 'Genomic Analysis of Congenital Diaphragmatic Hernia and Associated Congenital Anomalies',
    'attribution': 'https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs001110.v1.p1'
}
study_df = pd.DataFrame([study])
study_df.to_csv(os.path.join(DATA_DIR, 'study.txt'))

# Create investigator
invest = {
    'investigator_name': 'Wendy Chung',
    'institution': 'Columbia University Health Sciences'
}
inv_df = pd.DataFrame([invest])
inv_df.to_csv(os.path.join(DATA_DIR, 'investigator.txt'))

In [ ]:
@reformat_column_names
@dropna_rows_cols
def read_study_file_data(filepaths=None):
    """
    Read in raw study files
    """
    if not filepaths:
        filepaths = os.listdir(DBGAP_DIR)
        filepaths.extend(os.listdir(MANIFESTS_DIR))

    study_files = [{"study_file_name": f}
                   for f in filepaths]
    return pd.DataFrame(study_files)

@reformat_column_names
@dropna_rows_cols
def read_study_data(filepath=None):
    """
    Read study data
    """
    if not filepath:
        filepath = os.path.join(DATA_DIR,
                                'study.txt')
    df = pd.read_csv(filepath)

    return df

@reformat_column_names
@dropna_rows_cols
def read_investigator_data(filepath=None):
    """
    Read investigator data
    """
    if not filepath:
        filepath = os.path.join(DATA_DIR,
                                'investigator.txt')
    df = pd.read_csv(filepath)

    return df

@reformat_column_names
@dropna_rows_cols
def read_subject_data(filepath=None):
    """
    Read subject data file
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '4a_dbGaP_SubjectDS_corrected_7-16.xlsx')
    df = pd.read_excel(filepath, dtype={'SUBJECT_ID': str})
    # Decode consent ints to consent strings
    def func(row): 
        _map = {1: "Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)"}
        return _map.get(row['CONSENT'])
    df['CONSENT'] = df.apply(func, axis=1)

    # Decode affected status ints to strings
    def func(row): 
        _map = {0:'unknown', 1: "affected", 2: "unaffected"}
        return _map.get(row['AFFECTED_STATUS'])
    df['AFFECTED_STATUS'] = df.apply(func, axis=1)
    return df

@reformat_column_names
@dropna_rows_cols
def read_subject_attr_data(filepath=None):
    """
    Read subject attributes file
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '3a_dbGaP_SubjectAttributesDS_corrected.6.12.xlsx')
    df = pd.read_excel(filepath, dtype={'SUBJECT_ID': str})
    # Decode body_site chars to strings
    def func(row): 
        _map = {'B':'blood', 'SK': 'skin', 'D': 'diaphragm', 'SV': 'saliva', 'A': 'amniocytes', 'M': 'amniocytes'}
        return _map.get(row['body_site'])
    df['body_site'] = df.apply(func, axis=1)
    return df

@reformat_column_names
@dropna_rows_cols
def read_family_data(filepath=None):
    """
    Read pedigree data
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '6a_dbGaP_PedigreeDS_corrected.6.12.xlsx')
    df = pd.read_excel(filepath)
    del df['SEX']
    
    return df

@reformat_column_names
# @dropna_rows_cols
def read_sample_manifests():
    """
    Read and combine all sample manifest sheets
    """
    # Sample manifests
    # Combine all sample manifest sheets
    dfs = [pd.read_excel(os.path.join(MANIFESTS_DIR, filename))

           for filename in os.listdir(MANIFESTS_DIR)

           ]
    df = pd.concat(dfs)
    df = df[df['Sample ID'].notnull()]
    df.rename(columns={'Alias.2': 'is_proband'}, inplace=True)
    df['is_proband'] = df['is_proband'].apply(lambda x: True if x == 'Proband' else False)
    
    def func(row):
        val = str(row['Volume']).strip("uL")
        try:
            val = int(val)
        except ValueError:
            val = np.NaN
        return val
    
    df['Volume'] = df.apply(func, axis=1)
    
    def func(row):
        val = str(row['Concentration']).strip("ng/uL")
        try:
            val = int(val)
        except ValueError:
            val = np.NaN
        return val
    
    df['Concentration'] = df.apply(func, axis=1)
    
    df = df[pd.notnull(df.Concentration)]
    df = df[pd.notnull(df.Volume)]
    df.Concentration = df.Concentration.astype('int')
    df.Volume = df.Volume.astype('int')
    
    return df[['Concentration', 'Volume', 'Sample ID', 'Sample Type', 'is_proband']]

@reformat_column_names
@dropna_rows_cols
def read_subject_sample_data(filepath=None):
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '5a_dbGaP_SubjectSampleMappingDS cumulative.xlsx')
    return pd.read_excel(filepath, delimiter='\t')

@reformat_column_names
@dropna_rows_cols
def read_demographic_data(filepath=None):
    """
    Read demographic data from phenotype file
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '2a_dbGaP_SubjectPhenotypesDS.xlsx')
    df = pd.read_excel(filepath)
    # Make all values lower case
    for col in ['Ethnicity', 'Race']:
        df[col] = df[col].apply(lambda x: str(x).lower().strip())
        
    return df[['SUBJECT_ID', 'SEX', 'Ethnicity', 'Race']]

@reformat_column_names
@dropna_rows_cols
def read_phenotype_data(filepath=None):
    """
    Read phenotype file and insert HPO IDs
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '2a_dbGaP_SubjectPhenotypesDS.xlsx')

    df = pd.read_excel(filepath)
    df.drop(['Ethnicity', 'Race', 'SEX', 'discharge_status', 'ISOLATED'], inplace=True, axis=1)
    # Reshape to build the phenotypes df
    cols = df.columns.tolist()[1:]
    phenotype_df = pd.melt(df, id_vars='SUBJECT_ID', value_vars=cols,
                           var_name='phenotype', value_name='value')

    # Drop rows where value is NaN
    phenotype_df = phenotype_df[pd.notnull(phenotype_df['value'])]

    # Decode phenotypes to descriptive strings
    def func(row):
        _map = {0: 'no', 1: 'yes'}
        return _map.get(row['value'], row['value'])
    phenotype_df['value'] = phenotype_df.apply(func, axis=1)

    # Decode phenotypes to descriptive strings
    def func(row):
        # Always take most specific value
        if row['value'] not in ['yes', 'no']:
            val = row['value']
        else:
            _map = {'CHD': 'congenital heart defect', 'CNS': 'central nervous system defect', 
                'GI': 'gastrointestinal defect'}
            val = _map.get(row['phenotype'], 'congenital birth defect')
        return val
    phenotype_df['phenotype'] = phenotype_df.apply(func, axis=1)

    # Set observed
    phenotype_df['observed'] = phenotype_df['value'].apply(lambda x: 'positive' if x != 'no' else 'negative')
    del phenotype_df['value']
    
    # Add HPOs
    from dataservice.util.data_import.etl.hpo import mapper
    hpo_mapper = mapper.HPOMapper(DATA_DIR)
    phenotype_df = hpo_mapper.add_hpo_id_col(phenotype_df)
    
    # Add unique col
    def func(row): return "_".join(['phenotype', str(row.name)])
    phenotype_df['phenotype_id'] = phenotype_df.apply(func, axis=1)
    
    return phenotype_df

@reformat_column_names
@dropna_rows_cols
def read_outcome_data(filepath=None):
    """
    Read outcome data from phenotype file
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '2a_dbGaP_SubjectPhenotypesDS.xlsx')
    df = pd.read_excel(filepath)
    
    # Replace NaN values with None
    df['discharge_status'] = df['discharge_status'].where((pd.notnull(df['discharge_status'])),999)
    
    # Map discharge status
    # 1=Alive 4=Deceased 0=Fetal sample 8=unknown NA=Not applicable
    def func(row): 
        _map = {0:'alive', 1: 'deceased', 4:'fetal sample', 8: 'unknown'}
        return _map.get(int(row['discharge_status']), 'not applicable')
    df['discharge_status'] = df.apply(func, axis=1)
    
    # Add unique col
    def func(row): return "_".join(['outcome', str(row.name)])
    df['outcome_id'] = df.apply(func, axis=1)
    
    return df[['SUBJECT_ID', 'discharge_status', 'outcome_id']]

@reformat_column_names
@dropna_rows_cols
def read_genomic_file_manifest(filepath=None):
    """
    Read genomic file manifest (ties subjects to genomic files)
    """
    if not filepath:
        filepath = os.path.join(DATA_DIR, 'sample.txt')

    df = pd.read_csv(filepath, delimiter='\t')
    return df[['entity:sample_id', 'aligned_reads', 'crai_or_bai_path', 'cram_or_bam_path', 
               'library-1_name', 'library-2_name', 'max_insert_size', 'mean_depth', 'mean_insert_size', 
               'mean_read_length','min_insert_size', 'sample_alias', 'total_reads']]
    
HARMONIZED_TYPES = {'aligned reads'}
def read_genomic_files_info(filepath):
    """
    Read genomic file info json produced by Gen3 registration
    and convert into genomic file table for dataservice
    """
    data = read_json(filepath)
    df = pd.DataFrame(list(data.values()))

    # Reformat
    df['md5sum'] = df['hashes'].apply(lambda x: x['md5'])
    df['file_url'] = df['urls'].apply(lambda x: x[0])
    df['file_name'] = df['file_url'].apply(
        lambda file_url: os.path.basename(file_url))
    df['file_format'] = df['file_name'].apply(
        extract_uncompressed_file_ext)
    df.rename(columns={'did': 'latest_did', 'size': 'file_size'},
              inplace=True)

    # Data type
    def func(x):
        x = x.strip()
        if x.endswith('cram') or x.endswith('bam'):
            val = 'submitted aligned reads'
        elif x.endswith('crai'):
            val = 'submitted aligned reads index'
        elif 'fastq' in x:
            val = 'submitted reads'
        elif 'vcf' in x:
            val = 'simple nucleotide variation'
        else:
            val = None
        return val
    df['data_type'] = df['file_name'].apply(func)

    # Is harmonized
    df['is_harmonized'] = df['data_type'].apply(
        lambda data_type: True if data_type in HARMONIZED_TYPES else False)

    return df

## Explore

In [ ]:
# Db gap files
files = {f:os.path.join(DBGAP_DIR, f) for f in os.listdir(DBGAP_DIR)}
pprint(list(files.keys()))

### Subject 

In [ ]:
# Description of data
df1 = pd.read_excel(files['4b_dbGaP_SubjectDD_corrected_6_12.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df1

In [ ]:
# Subject data
df = pd.read_excel(files['4a_dbGaP_SubjectDS_corrected_7-16.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df.head()

In [ ]:
print(df['CONSENT'].unique())
print(df['AFFECTED_STATUS'].unique())
df.describe(include=['O']).T.sort_values('unique', ascending=False)

In [ ]:
# Subject attributes data description
df = pd.read_excel(files['3b_dbGaP_SubjectAttributesDD.xlsx'])
df

In [ ]:
# Subject attributes
df = pd.read_excel(files['3a_dbGaP_SubjectAttributesDS_corrected.6.12.xlsx'], delimiter='\t')
df.head()

In [ ]:
print(df.body_site.unique())
df.describe(include=['O']).T.sort_values('unique', ascending=False)

### Family/Pedigree

In [ ]:
# Data description
df = pd.read_excel(files['6b_dbGaP_PedigreeDD.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df.head()

In [ ]:
df = pd.read_excel(files['6a_dbGaP_PedigreeDS_corrected.6.12.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df.head()

In [ ]:
df.describe(include=['O']).T.sort_values('unique', ascending=False)

### Phenotypes

In [ ]:
# Data description
df = pd.read_excel(files['2b_dbGaP_SubjectPhenotypesDD.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df

In [ ]:
# Data
df = pd.read_excel(files['2a_dbGaP_SubjectPhenotypesDS.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df.head()

In [ ]:
df.describe(include=['O']).T.sort_values('unique', ascending=False)

### Samples

In [ ]:
# Subject sample data description
filepath = os.path.join(DBGAP_DIR, '5b_dbGaP_SubjectSampleMappingDD.xlsx')
df = pd.read_excel(filepath, delimiter='\t')
df.head()

In [ ]:
# Subject sample mapping
filepath = os.path.join(DBGAP_DIR, '5a_dbGaP_SubjectSampleMappingDS cumulative.xlsx')
df = pd.read_excel(filepath, delimiter='\t')
df.head()

In [ ]:
df.describe(include=['O']).T.sort_values('unique', ascending=False)

In [ ]:
# Sample manifests
# Combine all sample manifest sheets
dfs = [pd.read_excel(os.path.join(MANIFESTS_DIR, filename))

       for filename in os.listdir(MANIFESTS_DIR)

       ]
df = pd.concat(dfs)
df = df[df['Sample ID'].notnull()]
print(df.shape)
df.head()

In [ ]:
# Phenotype sheets
dfs = [pd.read_excel(os.path.join(MANIFESTS_DIR, filename), sheet_name=1)

       for filename in os.listdir(MANIFESTS_DIR)

       ]
df = pd.concat(dfs)
df = df[df['Sample ID'].notnull()]
# Make all values lower case
df['Primary Disease'] = df['Primary Disease'].apply(lambda x: str(x).lower())

print(df.shape)
df.head()

## Extract

#### Participants, Family Relationships

In [ ]:
subject_df = read_subject_data()
print(subject_df.nunique())
subject_df.head()

In [ ]:
subject_attr_df = read_subject_attr_data()
print(subject_attr_df.nunique())
subject_attr_df.head()

In [ ]:
family_df = read_family_data()
family_df.head()

In [ ]:
sample_manifest_df = read_sample_manifests()
print(sample_manifest_df.nunique())
sample_manifest_df.head()

In [ ]:
# Participant df
# Merge subject + subject attributes
df1 = pd.merge(subject_df, subject_attr_df, on='subject_id')
df1.head()

# Merge family
df2 = pd.merge(df1, family_df, on='subject_id')
print('{} Participants w/o samples merged'.format(df2.shape))

# Merge proband from sample manifests
participant_df = pd.merge(df2, sample_manifest_df[['sample_id', 'is_proband']], how='left', on='sample_id')
participant_df['is_proband'].fillna(False, inplace=True)
print('{} Participants w samples merged'.format(participant_df.shape))
participant_df.head()

#### Demographic

In [ ]:
demographic_df = read_demographic_data()
demographic_df = pd.merge(demographic_df, participant_df, on='subject_id')
demographic_df.head()
demographic_df.describe()

#### Samples

In [ ]:
sample_manifest_df.columns.tolist()

In [ ]:
# Subject sample mappings
subject_sample_df = read_subject_sample_data()
subject_sample_df.head()

# Merge with subject data
df3 = pd.merge(participant_df, subject_sample_df[['subject_id', 'sample_use']], on='subject_id')
print(df3.shape)

# Merge with sample manifests
sample_df = pd.merge(df3, sample_manifest_df, how='left', on='sample_id')
print(sample_df.shape)

#### Genomic Files, Seq Experiments

In [ ]:
# Sequencing experiments
gf_manifest_df = read_genomic_file_manifest()
print(gf_manifest_df.nunique())
gf_manifest_df.head()
# gf_manifest_df.describe(include='O')

In [ ]:
# Genomic file df
gf_file_info_df = read_genomic_files_info()
gf_file_info_df['subject_id'] = gf_file_info_df['file_name'].apply(lambda file_name: file_name.split('.')[0])
print(gf_file_info_df.shape)
gf_file_info_df.head()

In [ ]:
gf_file_info_df[['uuid', 'file_name', 'subject_id']].describe(include='O')

In [ ]:
# Merge with biospecimens
df = pd.merge(sample_df[['subject_id', 'sample_id']], gf_file_info_df, on='subject_id')
print(df.shape)
df.head()

In [ ]:
# Merge with sequencing experiments
genomic_file_df = pd.merge(df, gf_manifest_df, left_on='subject_id', right_on='sample_alias')
print(genomic_file_df.shape)
genomic_file_df.head()
# genomic_file_df.describe(include='O')

#### Phenotype

In [ ]:
# Read phenotype
phenotype_df = read_phenotype_data()
phenotype_df.head()

# Merge with participant df
phenotype_df = pd.merge(phenotype_df, participant_df, on='subject_id')
phenotype_df.head()

In [ ]:
phenos = phenotype_df[phenotype_df.observed == 'positive']
phenos.groupby(['phenotype']).count().sort_values(['phenotype_id'], ascending=False)

#### Outcomes

In [ ]:
outcome_df = read_outcome_data()
outcome_df.head()

In [ ]:
outcome_df = outcome_df[outcome_df.discharge_status != 'not applicable']
outcome_df.discharge_status.unique()
outcome_df.shape